##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - 3 Function calling.

Google released Gemma-3 its new variant that comes with Multimodality, Multilingual support upto 140 global languages, 128K context window and also Agentic capabilites like structured responses and function calling. In this notebook, we will explore how to use Function calling with open source models from HuggingFace via Gemma 3 4B-it model

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_3]Function_Calling_with_HF.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Gemma-3 setup

To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run
  the Gemma 3 4B-it model.
* Generate and configure a Kaggle username and an API key as Colab secrets.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.


### Install dependencies

Run the cell below to install all the required dependencies.

- Transformers: Latest version that supports Gemma3
- Accelerate
- DuckDuck Go Search - for function calling

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install accelerate

In [ ]:
!pip install duckduckgo-search

# Gemma 3

Google has released Gemma-3, a successor to its earlier Gemma models, focusing on open-source accessibility and multimodal vision capabilities. The model is designed to handle complex tasks with extended context lengths, support for multiple languages, and vision capabilities, making it suitable for a wide range of applications.

Gemma-3 has gained attention for outperforming notable models like DeepSeek V3, o3-mini, and Llama 3 405B, with extended context lengths, multilingual support, and multimodal capabilities.

- Extended context length from 32K to 128K using ROPE scaling (from 10K to 1M) on global self-attention layers.
- Available in 1B, 4B, 12B, and 27B sizes (Vision capability included, except for 1B, with bi-directional attention).
- Supports an extensive range of 140 global languages.
- Additional Features: Supports function calling, where the model can generate code or instructions to call external functions, and structured outputs, enabling it to produce data in specific formats like JSON or tables.

## Load Model weights

In [ ]:
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

In [ ]:
model_id = "google/gemma-3-4b-it"

In [ ]:
model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
torch.cuda.empty_cache()

## Function Calling Gemma

Function calling is a feature that allows large language models to interact with external systems, APIs, and tools in a structured way. This feature enables LLMs to not just generate text responses but to invoke specific functions when appropriate based on user queries. At its core, function calling involves describing functions to the LLM and having it intelligently return the function to call along with necessary arguments for particular tasks. For example, an LLM can be configured to retrieve stock price data by calling a specific function when a user asks about the current price of a particular stock.

In [ ]:
from duckduckgo_search import DDGS

def search(query:str):
  """
  search results to the user query

  Args:
      query: user prompt to fetch search results
  """
  req = DDGS()
  response = req.text(query,max_results=4)
  context = ""
  for result in response:
    context += result['body']
  return context

In [ ]:
tools = [search]

In [ ]:
query = "which teams played Cricket ICC Champions Trophy 2025 finals and who won?" # this is a realtime query, this event occured on March 9th 2025

## Define prompt for function calling

Function Calling follows these steps:

- Your application sends a prompt to the LLM along with function definitions
- The LLM analyzes the prompt and decides whether to respond directly or use defined functions
- If using functions, the LLM generates structured arguments for the function call
- Your application receives the function call details and executes the actual function
- The function results are sent back to the LLM
- The LLM provides a final response incorporating the function results

In [ ]:
conversation = [
    {
        "role": "system",
        "content": [{"type": "text", "text": """
        You are an expert search assistant. Use `search` to get the most accurate details.
        At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```. The python methods described below are imported and available,
        you can only use defined methods. The generated code should be readable and efficient. The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response.
        When using a ```tool_call``` think step by step why and how it should be used.

        The following Python methods are available:
        \`\`\`python
        def search(query:str):
          "
          search results to the user query

          Args:
             query: user prompt to fetch search results
          "
        \`\`\`

        User: \{user_message\}
        """}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": query}
        ]
    }
]

In [ ]:
inputs = processor.apply_chat_template(
            conversation,
            tools=tools,
            add_generation_prompt=True,
            return_dict=True,
            tokenize=True,
            return_tensors="pt",
).to(model.device, dtype=torch.bfloat16)

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=512)

In [ ]:
output = processor.decode(outputs[0], skip_special_tokens=True)

In [ ]:
response = output.split("model")[-1]

In [ ]:
print(response)


```tool_code
search(query="Which teams played in the Cricket ICC Champions Trophy 2025 final and who won?")
```


## Execute the function calling code

In [ ]:
def extract_tool_call(text):
    import io
    from contextlib import redirect_stdout

    pattern = r"```tool_code\s*(.*?)\s*```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        code = match.group(1).strip()
        # Capture stdout in a string buffer
        f = io.StringIO()
        with redirect_stdout(f):
            result = eval(code)
        output = f.getvalue()
        r = result if output == '' else output
        return f'```tool_output\n{r}\n```'''
    return None

In [ ]:
keyword = extract_tool_call(response)

In [ ]:
keyword

"```tool_output\nThe ninth edition of the Champions Trophy 2025 saw India being crowned as the winners on 9th March 2025 after they overcame New Zealand in the final. Several exceptional performers lit up the tournament with the bat and ball. The best of them made it to the Team of the Tournament. Here's what the side looks like: 1. Rachin Ravindra (New Zealand)The 2025 ICC Champions Trophy was the ninth edition of the ICC Champions Trophy, a quadrennial ODI cricket tournament organised by the International Cricket Council (ICC). In November 2021 as part of the 2024-2031 ICC men's hosts cycle, the ICC announced that the 2025 Champions Trophy would be played in Pakistan. [4]On 19 December 2024, following an agreement between Board of Control for ...Official ICC Champions Trophy, 2025 Cricket website - live matches, scores, news, highlights, commentary, rankings, videos and fixtures from the International Cricket Council. ... 2025. ICC announces Champions Trophy 2025 Team of the Tourname

In [ ]:
prompt = [{
        "role": "user",
        "content": [
            {"type": "text", "text": f"Results from the search:{keyword} User_query : {query}"}
        ]
    }
]

In [ ]:
final_prompt = processor.apply_chat_template(
            prompt,
            tools=tools,
            add_generation_prompt=True,
            return_dict=True,
            tokenize=True,
            return_tensors="pt",
).to(model.device, dtype=torch.bfloat16)

In [ ]:
response = model.generate(**final_prompt, max_new_tokens=512)

In [ ]:
print(processor.decode(response[0], skip_special_tokens=True))

user
Results from the search:```tool_output
The ninth edition of the Champions Trophy 2025 saw India being crowned as the winners on 9th March 2025 after they overcame New Zealand in the final. Several exceptional performers lit up the tournament with the bat and ball. The best of them made it to the Team of the Tournament. Here's what the side looks like: 1. Rachin Ravindra (New Zealand)The 2025 ICC Champions Trophy was the ninth edition of the ICC Champions Trophy, a quadrennial ODI cricket tournament organised by the International Cricket Council (ICC). In November 2021 as part of the 2024-2031 ICC men's hosts cycle, the ICC announced that the 2025 Champions Trophy would be played in Pakistan. [4]On 19 December 2024, following an agreement between Board of Control for ...Official ICC Champions Trophy, 2025 Cricket website - live matches, scores, news, highlights, commentary, rankings, videos and fixtures from the International Cricket Council. ... 2025. ICC announces Champions Troph